In [54]:
ls

archive.zip  spam.csv


In [55]:
cd//content/drive/MyDrive/Colab Notebooks/Assignment4

/content/drive/MyDrive/Colab Notebooks/Assignment4


In [56]:
pwd

'/content/drive/MyDrive/Colab Notebooks/Assignment4'

In [57]:
!unzip archive.zip

Archive:  archive.zip
replace spam.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


**Importing libraries**

In [58]:
import pandas as pd
import numpy as np
import tensorflow
import seaborn as sns
import matplotlib.pyplot as plt
import keras
import re
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

In [59]:
data=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Assignment4/spam.csv',delimiter=',', encoding="ISO-8859-1") 

In [60]:
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [61]:
data.shape

(5572, 5)

Preprocessing

In [62]:
data=data.drop(columns=["Unnamed: 2","Unnamed: 3","Unnamed: 4"])

In [66]:
data=data.rename({"v1":"Category","v2":"Message"},axis=1)

In [67]:
data.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [68]:
data["Message Length"]=data["Message"].apply(len)

In [69]:
ham_desc=data[data["Category"]=="ham"]["Message Length"].describe()
spam_desc=data[data["Category"]=="spam"]["Message Length"].describe()

In [70]:
data.describe(include="all")

,Category,Message,Message Length
count,5572,5572,5572.000000
unique,2,5169,NaN
top,ham,"Sorry, I'll call later",NaN
freq,4825,30,NaN
mean,NaN,NaN,80.118808
std,NaN,NaN,59.690841
min,NaN,NaN,2.000000
25%,NaN,NaN,36.000000
50%,NaN,NaN,61.000000
75%,NaN,NaN,121.000000


In [71]:
data["Category"].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

**Our data is unbalance since ham containing 4825 data and spam contains 747 data so we perform undersampling**

In [72]:
minority_len=len(data[data["Category"]=="spam"])
majority_len=len(data[data["Category"]=="ham"])

In [73]:
minority_indices=data[data["Category"]=="spam"].index
majority_indices=data[data["Category"]=="ham"].index

In [74]:
random_majority_indices=np.random.choice(majority_indices,size=minority_len,replace=False)

In [75]:
undersampled_indices=np.concatenate([minority_indices,random_majority_indices])

In [76]:
df=data.loc[undersampled_indices]

In [77]:
df=df.sample(frac=1)

In [78]:
df=df.reset_index()

In [79]:
df=df.drop(columns=["index"],)

We balanced our ham and spam data

In [80]:
df["Category"].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

In [81]:
df["Label"]=df["Category"].map({"ham":0,"spam":1})

In [82]:
df.head()

,Category,Message,Message Length,Label
0,ham,I hope your pee burns tonite.,29,0
1,ham,Yes..he is really great..bhaji told kallis bes...,104,0
2,ham,You are right. Meanwhile how's project twins c...,53,0
3,ham,Okie,4,0
4,spam,"Shop till u Drop, IS IT YOU, either 10K, 5K, å...",158,1


In [91]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [92]:
import nltk

In [93]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [84]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
stemmer=PorterStemmer()

**Create model**

In [94]:
    #declare empty list to store tokenized message
corpus=[]

#iterate through the df["Message"]
for message in df["Message"]:

    
    #replace every special characters, numbers etc.. with whitespace of message
    #It will help retain only letter/alphabets
  message=re.sub("[^a-zA-Z]"," ",message)
    
    #convert every letters to its lowercase
  message=message.lower()
  message=message.split()
    
    #perform stemming using PorterStemmer for all non-english-stopwords
  message=[stemmer.stem(words)
            for words in message
             if words not in set(stopwords.words("english"))
            ]
    #join the word lists with the whitespace
  message=" ".join(message)
    
    #append the message in corpus list
  corpus.append(message)

In [95]:
from tensorflow.keras.preprocessing.text import one_hot
vocab_size=10000
oneHot_doc=[one_hot(words,n=vocab_size)
           for words in corpus]

In [96]:
df["Message Length"].describe()

count    1494.000000
mean      104.698795
std        59.898041
min         2.000000
25%        48.000000
50%       116.000000
75%       153.000000
max       910.000000
Name: Message Length, dtype: float64

In [97]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
sentence_len=200
embedded_doc=pad_sequences(oneHot_doc,maxlen=sentence_len,padding="pre")

In [98]:
extract_features=pd.DataFrame(data=embedded_doc)
target=df["Label"]

In [99]:
df_final=pd.concat([extract_features,target],axis=1)

In [101]:
df_final.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,Label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,4861,404,9972,4351,0
1,0,0,0,0,0,0,0,0,0,0,...,8189,3447,1331,1312,5827,716,8944,7441,2884,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,4398,9452,9413,3703,4521,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,9254,0
4,0,0,0,0,0,0,0,0,0,0,...,2809,7259,5468,8925,7963,8684,3577,6095,4151,1


In [102]:
#split the dataframe into dependent and independent variables
X=df_final.drop("Label",axis=1)
y=df_final["Label"]

In [103]:
from sklearn.model_selection import train_test_split

In [104]:
X_trainval,X_test,y_trainval,y_test=train_test_split(X,y,random_state=42,test_size=0.15)

In [105]:
X_train,X_val,y_train,y_val=train_test_split(X_trainval,y_trainval,random_state=42,test_size=0.15)

Build the model

In [106]:
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential

In [107]:
model=Sequential()

In [108]:
feature_num=100
model.add(Embedding(input_dim=vocab_size,output_dim=feature_num,input_length=sentence_len))
model.add(LSTM(units=128))
model.add(Dense(units=1,activation="sigmoid"))

Compile the model

In [109]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.001),loss="binary_crossentropy",metrics=["accuracy"])

fit the model

In [110]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10)

Epoch 1/10
34/34 [==============================] - 16s 382ms/step - loss: 0.5756 - accuracy: 0.6809 - val_loss: 0.5194 - val_accuracy: 0.7906
Epoch 2/10
34/34 [==============================] - 12s 352ms/step - loss: 0.2860 - accuracy: 0.9341 - val_loss: 0.1676 - val_accuracy: 0.9424
Epoch 3/10
34/34 [==============================] - 13s 374ms/step - loss: 0.0807 - accuracy: 0.9759 - val_loss: 0.1168 - val_accuracy: 0.9581
Epoch 4/10
34/34 [==============================] - 12s 348ms/step - loss: 0.0375 - accuracy: 0.9898 - val_loss: 0.1318 - val_accuracy: 0.9581
Epoch 5/10
34/34 [==============================] - 14s 422ms/step - loss: 0.0248 - accuracy: 0.9935 - val_loss: 0.1299 - val_accuracy: 0.9581
Epoch 6/10
34/34 [==============================] - 12s 357ms/step - loss: 0.0273 - accuracy: 0.9954 - val_loss: 0.1074 - val_accuracy: 0.9634
Epoch 7/10
34/34 [==============================] - 14s 409ms/step - loss: 0.0217 - accuracy: 0.9954 - val_loss: 0.1027 - val_accuracy: 0.9581

In [119]:
y_pred=model.predict(X_test)
y_pred=(y_pred>0.5)

8/8 [==============================] - 1s 98ms/step


test the model

In [120]:
from sklearn.metrics import accuracy_score,confusion_matrix

In [121]:
score=accuracy_score(y_test,y_pred)
print("Test Score:{:.2f}%".format(score*100))

Test Score:96.44%


In [132]:
def classify_message(model,message):
    for sentences in message:
        sentences=nltk.sent_tokenize(message)
        for sentence in sentences:
            #replace all special characters
            words=re.sub("[^a-zA-Z]"," ",sentence)
            if words not in set(stopwords.words('english')):
                word=nltk.word_tokenize(words)
                word=" ".join(word)
        oneHot=[one_hot(word,n=vocab_size)]
    text=pad_sequences(oneHot,maxlen=sentence_len,padding="pre")
    predict=model.predict(text)
    
    #if predict value is greater than 0.5 its a spam
    if predict>0.5:
        print("It is a spam")
    #else the message is not a spam    
    else:
        print("It is not a spam")

In [133]:
message1="I am having a bad day and I would like to have a break today"
message2="This is to inform you had won a lottery and the subscription will end in a week so call us."

In [134]:
import nltk

In [135]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [136]:
classify_message(model,message1)

1/1 [==============================] - 0s 44ms/step
It is not a spam


In [137]:
classify_message(model,message2)

1/1 [==============================] - 0s 28ms/step
It is a spam
